In [ ]:
print("Running")

In [ ]:
import sys
import gym
import glob
import torch
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("../")
from ACDir import ActorCriticDiscrete

In [ ]:
deg2rad = lambda x: x * 3.1415926536 / 180.
rad2deg = lambda x: x * 180. / 3.1415926536

In [ ]:
model = ActorCriticDiscrete(num_inputs=1, num_actions=3)
model.load_state_dict(torch.load("../weights/mfmaac_epochs800_measure500_timesteps10_20210210-123802.pt"))#"../weights/mfmaac_epochs800_measure500_timesteps10_20210216-213727.pt"))
model.eval()

In [ ]:
values = []
ran = list(range(-90,270))
for Delta in ran:
    Delta = torch.tensor(deg2rad(Delta)).reshape((1,)).float()
    Delta = torch.nn.functional.relu(model.affine(Delta))
    state_value = model.value_layer(Delta)
    values.append(state_value)

plt.plot(ran, values)
#env.load(glob.glob("../runs/mfmaac*")[-1])

In [ ]:
for Delta in ran[90:90+180:15]:
    Delta_orig = Delta
    Delta = torch.tensor(deg2rad(Delta)).reshape((1,)).float()
    Delta = torch.nn.functional.relu(model.affine(Delta))
    action_probs = torch.nn.functional.softmax(model.action_layer(Delta))    
    #print([-1, 0, 1], action_probs)
    fig, ax = plt.subplots()
    ax.set_title(f'Delta = {Delta_orig}°')
    ax.bar([-1, 0, 1], action_probs.detach().numpy(), )
    plt.show()

In [ ]:
actions = []

for Delta in ran[90:90+180]:
    Delta_orig = Delta
    Delta = torch.tensor(deg2rad(Delta)).reshape((1,)).float()
    Delta = torch.nn.functional.relu(model.affine(Delta))
    action_probs = torch.nn.functional.softmax(model.action_layer(Delta))    
    #print()
    actions.append((action_probs * torch.tensor([-1, 0, 1]) * 3).mean().detach())

plt.plot(ran[90:90+180], actions)